# PYNQ with Ardumoto motor driver shield 

Actuators such as D.C. Motors are one of the most useful peripherals in most of the projects. 
PYNQ can be interfaced with Sparkfun Ardumoto Shield to control two motors simultaneously. 

## Contents

* [Background](#Background)
* [Ardumoto Shield](#Ardumoto-Shield)
* [Connections](#Connections)
* [Setup](#Setup)
* [Motor Control](#Motor-Control)
* [Robotics Concepts](#Robotics-Concepts)


## Background

The simplest of all motors, DC motors turn when a DC voltage is applied across it. This kind of motor can be found in drones, power tools, and robots. A DC motor can change speend and direction depending on how much power is fed to it and in which direction.

The DC motor uses a magnetic field generated by the by an electromagnet to turn the armature of a motor. The electromagnet is activated by applying voltage, so when the power is on, the magnetic field it generates will cause the armature (a coil of wire) to generate its own nagmetic field, these fields push eachother away and cause the armature to spin.

To get the motor to spin the other way, we need to reverse the applied voltage, meaning the flow of current through the motor will be reversed. Unfortunately switching the direction of current from a controller is difficult. The processors use low current and voltages, plus they are usually disconnected from the motor to prevent inductive feedback distrupting their operation.

If only there was some kind of device that can help us control the power we supply to our DC motors…

##### How H-Bridges Work
An H-Bridge is a circuit that allows voltages to be applied across a load in either direction. Electric current flows from the source to ground, and many components need to be oriented according to the direction of current to work as expected. An H-Bridge is a circuit built to change the direction of the voltage and thus the current flowing to a load.

An H-Bridge is made up of four switches: two in series, and two in parallel, with the load placed in between the switches. In this configuration the circuit takes an “H” shape.

![alt text](data/H-Bridge.gif)

In order to change the direction of the voltage supplied, the H-Bridge controls the switches that deliver power to the load (M). 

Looking at the diagram, if we close S1 and S4 while leaving the rest open, the voltage will be applied from left to right across the motor. 

If S2 and S3 are closed instead and the others open, the voltage will be applied from right to left.

## Ardumoto Shield

[Ardumoto](https://www.sparkfun.com/products/14180).
Ardumoto supports upto two DC motor.
It uses H-Bridge to control the speed and direction of each motor.

Ardumoto shield sits on top of the arduino headers on the PYNQ board and uses an external battery to power the motors.

The Ardumoto shield can be mounted on the PYNQ Board similar to PYNQ arduino shield:

Motor A and Motor B are connected as below to the arduino pins:

| Pin number | Functionality                    |
|------------|----------------------------------|
| 2          | Direction control for motor A    |
| 3          | PWM control (speed) for motor A  |
| 4          | Direction control for motor B    |
| 11         | PWM control (speed) for motor B  |

### Connections

#### Connecting Ardumot to PYNQ Board

![alt text](data/PYNQ_Ardumoto.jpg)

#### Connecting the Motors to Ardumoto Shield  

![alt text](data/ardumoto_wiring_cdc.jpg)


## Setup

This training demonstrates how C code can be invoked from within the Python notebook to control a peripheral.
But before we start exploring that, as always we need to make sure the base overlay is loaded to the board.

We will also define some global constants that will be useful to control the motor.


In [14]:
# Setup base PYNQ environment
from pynq.overlays.base import BaseOverlay

#Vikhyat, where is the base.bit file stored on the SD card? 
base = BaseOverlay("base.bit") 

# Import sleep
from time import sleep

# Define constants to control motor direction
MOTOR_A = 0
MOTOR_B = 1
POLAR_DEFAULT = 0
POLAR_REVERSE = 1
CLOCKWISE = 0
ANTICLOCKWISE = 1

In [15]:
%%microblaze base.ARDUINO
//include supporting libraries
#include "xio_switch.h"
#include "gpio.h"
#include "timer.h"

#define DEFAULT_PERIOD 625998
#define DEFAULT_DUTY 312998

// define hardware pins we would use for controlling the shield, that controls the motors
#define DIR_A_PIN 2
#define PWM_A_PIN 3
#define DIR_B_PIN 4
#define PWM_B_PIN 11

typedef enum motor {
    MOTOR_A = 0,
    MOTOR_B = 1,
}motor_e;

static unsigned int pol_a = 0, pol_b = 0;
static unsigned int dir_a = 0, dir_b = 0;
static unsigned int duty_a = 50, duty_b = 50;

static timer timer_a;
static timer timer_b;
static gpio gpio_a;
static gpio gpio_b;

// helper function we need to control the shield 
unsigned int init_ardumoto(){    
    timer_a = timer_open_device(0);
    timer_b = timer_open_device(5);
    set_pin(PWM_A_PIN, PWM0);
    set_pin(PWM_B_PIN, PWM5);
    gpio_a = gpio_open(DIR_A_PIN);
    gpio_b = gpio_open(DIR_B_PIN);
    gpio_set_direction(gpio_a, GPIO_OUT);
    gpio_set_direction(gpio_b, GPIO_OUT);
    return 0;
}


void configure_polar(unsigned int motor, unsigned int polarity){
    if (motor == MOTOR_A) {
        pol_a = polarity;
    }else if (motor == MOTOR_B) {
        pol_b = polarity;
    }
}

// function to set the direction of the motor, clockwise or anticlockwise
void set_direction(unsigned int motor, unsigned int direction){
    if (motor == MOTOR_A){
        dir_a = (direction)? pol_a : !pol_a;
    }
    else if (motor == MOTOR_B){
        dir_b = (direction)? pol_b : !pol_b;
    }
}

// function to control the speed of the motors; the speed range is 1-100; the speed is controlled using PWM signal
void set_speed(unsigned int motor, unsigned int speed){
    if (motor == MOTOR_A) {
        duty_a = speed;
    } else if (motor == MOTOR_B) {
        duty_b = speed;
    }
}

// run the motors with the previously configured direction and speed
void run(unsigned int motor){
    if (motor == MOTOR_A) {
        gpio_write(gpio_a, dir_a);
        timer_pwm_generate(timer_a, DEFAULT_PERIOD, 
                           duty_a*DEFAULT_PERIOD/100);
    }else if(motor == MOTOR_B) {
        gpio_write(gpio_b, dir_b);
        timer_pwm_generate(timer_b, DEFAULT_PERIOD, 
                           duty_b*DEFAULT_PERIOD/100);
    }
}

// stop the PWM signal and 'brake' the motor
void stop(unsigned int motor){
    if (motor == MOTOR_A) {
        timer_pwm_stop(timer_a);
    }else if (motor == MOTOR_B){
        timer_pwm_stop(timer_b);
    }
}

In [16]:
# initialize the ardumoto shield
init_ardumoto()
configure_polar(MOTOR_A, POLAR_DEFAULT) 
configure_polar(MOTOR_B, POLAR_DEFAULT)

## Motor Control

In [17]:
from time import sleep
#Function to run motor in a given direction at given speed for a given duration 
def run_motor(motor, direction, speed, duration):
    set_direction(motor,direction)
    set_speed(motor,speed)
    run(motor)
    sleep(duration)
    stop(motor)
    sleep(1)
    
#Function to run both motor in a given direction at given speed for a given duration 
def run_BothMotors(direction_a, speed_a,direction_b, speed_b, duration):
    set_direction(MOTOR_A,direction_a)
    set_speed(MOTOR_A,speed_a)
    set_direction(MOTOR_B,direction_b)
    set_speed(MOTOR_B,speed_b)
    run(MOTOR_A)
    run(MOTOR_B)
    sleep(duration)
    stop(MOTOR_A)
    stop(MOTOR_B)
    sleep(1)

### Run both motors clockwise at low speed

Lets use Ardumoto shield to run both motors clockwise at low speed (10%)

In [18]:
set_direction(MOTOR_A, CLOCKWISE)
set_direction(MOTOR_B, CLOCKWISE)
set_speed(MOTOR_A, 10)
set_speed(MOTOR_B, 10)

print('Running Motors A and B at 10% speed for 3 seconds')
run(MOTOR_A)
run(MOTOR_B)

# suspend the current thread for 3 seconds
sleep(3)

print('Stopping Motors A and B')
stop(MOTOR_A)
stop(MOTOR_B)
sleep(3)

Running Motors A and B at 10% speed for 3 seconds
Stopping Motors A and B


### Run both motors counterclockwise one after the other at low speed

Lets use Ardumoto shield to run both motors counterclockwise at low speed (10%)

In [19]:
print('Running Motors A at 10% speed for 3 seconds')
run_motor(MOTOR_A,ANTICLOCKWISE,10,3)
print('Running Motors B at 10% speed for 3 seconds')
run_motor(MOTOR_B,ANTICLOCKWISE,10,3)
print('Motors Stopped')

Running Motors A at 10% speed for 3 seconds
Running Motors B at 10% speed for 3 seconds
Motors Stopped


### Run both motors clockwise at high speed

Lets use Ardumoto shield to run both motors clockwise at high speed (90%)


In [20]:
print('Running Motors A and B at 90% speed for 3 seconds in clockwise direction')
run_BothMotors(CLOCKWISE,90,CLOCKWISE,90,3)

Running Motors A and B at 90% speed for 3 seconds in clockwise direction


## Integrating with Sparkfun Environment Sensor

Let us now control Motor speed as a function of temperature sensed by Sparkfun Environment Sensor.

The Envrioment sensor uses a Qwiic-to-Grove cable to talk to PYNQ Z2 board over I2C bus. 

#### Sparkfun Environment Sensor
![alt text](data/Sparkfun_Env.jpg)

#### Qwiic-Grove Cable
![alt text](data/Qwiic_grove.jpg)

<div class="alert alert-box alert-warning"><ul>
    <h4 class="alert-heading">Make Physical Connections </h4>
    <li>Connect the PYNQ Grove Adapter to PMODB connector.  Connect Qwiic side of cable to Sparkfun Environment Sensor and the grove side to the G3 connector of the Adapter.</li></ul>
</div>

### Adapter configuration

In [21]:
from pynq_peripherals import ArduinoSEEEDGroveAdapter, PmodGroveAdapter
adapter=PmodGroveAdapter(base.PMODB, G3='grove_envsensor')
envsensor=adapter.G3

### Configure and read sensor data

In [22]:
# Initialize the sensor 
if (not envsensor.init()):
    print("Sensor configuration failed")
#read temperature
temp = envsensor.read_temperature() 
#read pressure
pres = envsensor.read_pressure() 
#read humidity
humd = envsensor.read_humidity() 
print(f"Temperature = {temp} °C, Pressure = {pres/100} hPa, Humidity = {humd} % r.H.")

Temperature = 27.31999969482422 °C, Pressure = 847.47 hPa, Humidity = 51.064998626708984 % r.H.


### Control Motor according to temperature

In [26]:
# Please set switch 0 low to start monitoring
Threshold = 31 # °C
fan_speed = 30
fan_duration = 3
while(base.switches[0].read()) :
    temp = envsensor.read_temperature()
    sleep(0.5)
    print(f"Temperature = {temp} °C")
    if temp >= Threshold:
        print(f"Threshold reached!!! Running fan for 3 seconds")
        run_motor(MOTOR_A,ANTICLOCKWISE,fan_speed,fan_duration)
        break

Temperature = 27.34000015258789 °C
Temperature = 27.639999389648438 °C
Temperature = 27.75 °C
Temperature = 27.799999237060547 °C
Temperature = 27.829999923706055 °C
Temperature = 27.860000610351562 °C
Temperature = 27.969999313354492 °C
Temperature = 29.170000076293945 °C
Temperature = 30.09000015258789 °C
Temperature = 30.5 °C
Temperature = 30.719999313354492 °C
Temperature = 30.899999618530273 °C
Temperature = 31.030000686645508 °C
Threshold reached!!! Running fan for 3 seconds


### Robotics Concepts 

A D.C. Motor assembly can be used with a chassis to drive and steer a robot in different directions.
Depending on the relative speed and direction of the two motors, we can achive different angles of turn.

Let us first define some helper functions for this:

* Move Forwards
* Move Backwards
* Turn left
* Turn Right


In [11]:
def robot_GoForward (speed,time):
    set_direction(MOTOR_A, CLOCKWISE)
    set_direction(MOTOR_B, CLOCKWISE)
    set_speed(MOTOR_A, speed)
    set_speed(MOTOR_B, speed)
    run(MOTOR_A)
    run(MOTOR_B)
    sleep(time)
    stop(MOTOR_A)
    stop(MOTOR_B)
    sleep(1)
    
def robot_GoBackward (speed,time):
    set_direction(MOTOR_A, ANTICLOCKWISE)
    set_direction(MOTOR_B, ANTICLOCKWISE)
    set_speed(MOTOR_A, speed)
    set_speed(MOTOR_B, speed)
    run(MOTOR_A)
    run(MOTOR_B)
    sleep(time)
    stop(MOTOR_A)
    stop(MOTOR_B)
    sleep(1)
    
def robot_TurnLeft (speed,time):
    set_direction(MOTOR_A, CLOCKWISE)
    set_direction(MOTOR_B, ANTICLOCKWISE)
    set_speed(MOTOR_A, speed)
    set_speed(MOTOR_B, speed)
    run(MOTOR_A)
    run(MOTOR_B)
    sleep(time)
    stop(MOTOR_A)
    stop(MOTOR_B)
    sleep(1)
    
def robot_TurnRight (speed,time):
    set_direction(MOTOR_A, ANTICLOCKWISE)
    set_direction(MOTOR_B, CLOCKWISE)
    set_speed(MOTOR_A, speed)
    set_speed(MOTOR_B, speed)
    run(MOTOR_A)
    run(MOTOR_B)
    sleep(time)
    stop(MOTOR_A)
    stop(MOTOR_B)
    sleep(1)
    
def robot_FollowPath (*path_tuples):
    for t in path_tuples:
        if t[0] == 'f':
            robot_GoForward(t[1],t[2])
        elif t[0] == 'b':
            robot_GoBackward(t[1],t[2])
        elif t[0] == 'r':
            robot_TurnRight(t[1],t[2])
        elif t[0] == 'l':
            robot_TurnLeft(t[1],t[2])
        else:
            print(f"Unknown Direction: {t[0]}")

### Let us learn to maneuver our robot
#### The animation below demonstrates the robot taking 3 forward steps, 2 steps to the right, and 2 steps down (forward)

![alt text](data/move_1.gif)

### Each helper function takes two arguments, speed and time. You may select any speed you like; the time corresponds to the steps we want to advance. For example, robotGoForward (10,3) will run the motor at 10% of the speed for 3 seconds. To keep it simple, in our case, the time corresponds to the steps, 3 steps in this example.

### Note: The robot will make a left or right depending on where the motor, A and B, are fixed to the robot. If you see it makes a left instead of a right, swap the motors. 

In [12]:
robot_GoForward (10,3)
robot_TurnRight (5, 1)
robot_GoForward (10,2)
robot_TurnRight (5, 1)
robot_GoForward (10,2)

#### The animation below demonstrates the robot taking 3 forward steps, 2 steps to the left, 2 steps to the right, 2 steps to the left, and 4 steps reverse 

![alt text](data/move_2.gif)

In [13]:
path = [('f',10,3),('l',10,1),('f',10,2),('r',10,1),('f',10,2),('l',10,1),('f',10,2)]
robot_FollowPath(*path)
# We take the reverse move to demonstrate a situation where we do not want to turn around the robot to face the reverse direction; 
# something similar to a 'R' gear in automobiles
robot_GoBackward (10,4)

### Connecting Battery to Ardumoto

In some robotics projects, it might be required to power Motors and PYNQ Z2 board using a battery. 

This can be done using the barrel connector as shown below.

#### Connecting the Battery to Ardumoto Shield 

![alt text](data/Ardumoto_Battery.jpg)